---

# Stacked_Bidirectional_Unidirectional_LSTM

---

In [13]:
!pip install keras==2.1.5
# 수행이후 런타임 재시작

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# 개인 workspace path설정 필요
base_path = '/content/drive/MyDrive/workspace/attention-based-lstm'

In [16]:
import os
if not os.path.exists(base_path):
  !mkdir $base_path
%cd $base_path
if not os.path.exists(base_path+'/Stacked_Bidirectional_Unidirectional_LSTM'):
  !git clone https://github.com/zhiyongc/Stacked_Bidirectional_Unidirectional_LSTM.git
%cd ./Stacked_Bidirectional_Unidirectional_LSTM/Keras

/content/drive/MyDrive/workspace/attention-based-lstm
/content/drive/MyDrive/workspace/attention-based-lstm/Stacked_Bidirectional_Unidirectional_LSTM/Keras


In [17]:
!cp -f ../../callback.py /usr/local/lib/python3.10/dist-packages/keras/callbacks.py
!cp -f ../../tensorflow_backend.py /usr/local/lib/python3.10/dist-packages/keras/backend/tensorflow_backend.py
!cp -f ../../optimizers.py /usr/local/lib/python3.10/dist-packages/keras/optimizers.py
!cp -f ../../merge.py ./merge.py

In [18]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Embedding, Input
from keras.layers import LSTM, SimpleRNN, GRU, Masking #Merge, merge
from keras.layers import concatenate, Concatenate
from keras.models import Model
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.layers.wrappers import Bidirectional

import numpy as np
import pandas as pd
from numpy.random import RandomState
from random import shuffle
import datetime, itertools, warnings
from merge import *

np.random.seed(1024)
warnings.filterwarnings(action='ignore', message='Setting attributes')

In [19]:
# Test case
csv_cases = ['pre_price', 'pre_price_s_7d','pre_price_s_14d', 'pre_price_s_21d', 'pre_price_s_28d']
window_cases = [44, 24, 44, 28, 135, 81, 145, 89, 89, 52, 94, 59, 88, 47, 88, 57, 79, 39, 74, 45]  #42
# horizon_cases = [a+1 for a in range(7)] + [14]
horizon_cases = [1,2,3]

all_cases = list(itertools.product(*[csv_cases,window_cases,horizon_cases]))
print("경우의 수 : %s개 | ex:" %len(all_cases), all_cases[0])

경우의 수 : 300개 | ex: ('pre_price', 44, 1)


In [20]:
for i in all_cases :
  print('#'*50)
  print(i)
  Y_test_scale, Y_pred_test, X_max = stacked_lstm(
      file_name = '/content/drive/MyDrive/workspace/data/'+str(i[0])+'.csv',
      model_epoch = 20,
      horizon = 1,
      window_size = 30)

  df_real = pd.DataFrame(np.array(Y_test_scale*X_max), columns=['potato', 'lettuce', 'onion', 'cucumber', 'high_tmp', 'rain_amt', 'avg_humid', 'avg_tmp', 'low_tmp', 'dtt', 'calweek'])
  df_pred = pd.DataFrame(np.array(Y_pred_test*X_max), columns=['potato', 'lettuce', 'onion', 'cucumber', 'high_tmp', 'rain_amt', 'avg_humid', 'avg_tmp', 'low_tmp', 'dtt', 'calweek'])
  #df_real.to_csv(f'../../f_{i[0]}_w_{i[1]}_h_{i[2]}_real.csv')
  #df_pred.to_csv(f'../../f_{i[0]}_w_{i[1]}_h_{i[2]}_pred.csv')
  break

##################################################
('pre_price', 44, 1)


/content/drive/MyDrive/workspace/attention-based-lstm/Stacked_Bidirectional_Unidirectional_LSTM/Keras/merge.py:53: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(30, 11), units=11)`
  model.add(LSTM(output_dim = X.shape[2], return_sequences=True, input_shape = (X.shape[1], X.shape[2])))
/content/drive/MyDrive/workspace/attention-based-lstm/Stacked_Bidirectional_Unidirectional_LSTM/Keras/merge.py:54: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=False, input_shape=(30, 11), units=11)`
  model.add(LSTM(output_dim = X.shape[2], return_sequences=False, input_shape = (X.shape[1], X.shape[2])))
/usr/local/lib/python3.10/dist-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 4648 samples, validate on 1163 samples
Epoch 1/20
4648/4648 [==============================] - 16s 3ms/step - loss: 0.0112 - val_loss: 0.0010
Epoch 2/20
4648/4648 [==============================] - 8s 2ms/step - loss: 5.4270e-04 - val_loss: 7.4425e-04
Epoch 3/20
4648/4648 [==============================] - 7s 1ms/step - loss: 3.8981e-04 - val_loss: 8.5104e-04
Epoch 4/20
4648/4648 [==============================] - 8s 2ms/step - loss: 3.0707e-04 - val_loss: 5.5003e-04
Epoch 5/20
4648/4648 [==============================] - 7s 1ms/step - loss: 2.6784e-04 - val_loss: 5.6967e-04
Epoch 6/20
4648/4648 [==============================] - 8s 2ms/step - loss: 2.4178e-04 - val_loss: 4.4410e-04
Epoch 7/20
4648/4648 [==============================] - 6s 1ms/step - loss: 2.2783e-04 - val_loss: 4.0404e-04
Epoch 8/20
4648/4648 [==============================] - 8s 2ms/step - loss: 2.0962e-04 - val_loss: 3.3671e-04
Epoch 9/20
4648/4648 [==============================] - 7s 1ms/step - loss: 1.9

In [ ]:
df_real

,potato,lettuce,onion,cucumber,high_tmp,rain_amt,avg_humid,avg_tmp,low_tmp,dtt,calweek
0,24000.0,30000.0,12000.0,28000.0,0.3,0.1,64.4,-2.2,-6.5,6.8,202001.0
1,24000.0,30000.0,12000.0,28000.0,3.8,0.0,65.4,1.0,-0.7,4.5,202001.0
2,24000.0,30000.0,12750.0,30000.0,4.6,0.0,56.9,-0.1,-3.4,8.0,202001.0
3,24000.0,30000.0,12750.0,30000.0,6.1,0.0,50.8,1.2,-2.8,8.9,202001.0
4,24000.0,30000.0,12750.0,30000.0,6.6,0.0,45.6,1.3,-3.2,9.8,202001.0
...,...,...,...,...,...,...,...,...,...,...,...
817,43000.0,17700.0,8625.0,57000.0,15.1,0.0,35.6,8.2,1.5,13.6,202213.0
818,52300.0,17000.0,9975.0,57700.0,15.6,0.0,52.9,9.7,3.3,12.3,202213.0
819,52300.0,18000.0,9975.0,49700.0,15.7,0.4,76.5,10.8,5.9,9.8,202213.0
820,60000.0,20500.0,9975.0,46200.0,14.7,0.0,63.5,10.9,7.6,7.1,202213.0


In [ ]:
df_pred

,potato,lettuce,onion,cucumber,high_tmp,rain_amt,avg_humid,avg_tmp,low_tmp,dtt,calweek
0,24330.169922,19737.158203,9293.804688,25183.111328,-711.881042,541.403137,-473.079773,438.563843,-911.171631,329.789642,199511.921875
1,24427.289062,21960.347656,9164.588867,25765.734375,-834.852722,487.743164,-563.065491,290.747498,-999.052612,218.913742,199596.625000
2,24630.011719,23738.406250,9229.181641,25953.199219,-875.966431,445.229797,-532.717957,304.099030,-1003.874512,202.665771,199555.609375
3,24824.875000,24914.033203,9424.540039,25784.285156,-869.771301,422.415283,-453.524048,388.805969,-980.626099,240.321838,199456.921875
4,24875.744141,25844.433594,9701.958008,26052.164062,-800.513855,410.012939,-335.559937,458.249512,-925.555176,304.912231,199450.437500
...,...,...,...,...,...,...,...,...,...,...,...
817,42106.941406,15847.959961,6526.216309,52750.414062,-868.643250,2.910383,-1136.088867,607.333252,-458.699982,-91.095695,199755.203125
818,42517.437500,15926.161133,6790.676758,53998.546875,-946.388550,-53.928524,-1050.468506,813.321106,-467.145996,-66.959900,199354.953125
819,43104.140625,15268.890625,7191.656250,53272.253906,-1044.484253,-52.677666,-923.305847,1055.870850,-506.685333,-15.897688,198826.640625
820,44332.953125,15087.054688,7139.331055,53756.187500,-1089.882080,-209.055283,-618.291565,1292.411499,-452.705200,40.444790,198542.203125
